In [1]:
import pandas as pd
import datetime
import numpy as np
from scipy import stats
pd.set_option('float_format', '{:f}'.format)

In [2]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
df = pd.read_csv("trade_new.csv", delimiter=',', parse_dates=['sldatime'], date_parser=dateparse).fillna(-1)
df = df.sort_values('sldatime')
# train
# directory = "train"
# feature_extract_df = df.copy().query('20160201 <= sldatime < 20160601')
# last_week_df = df.copy().query('20160525 <= sldatime < 20160601')
# last_month_df = df.copy().query('20160501 <= sldatime < 20160601')
# label_df = df.copy().query('20160601 <= sldatime < 20160701')
# feature_extract_df["just_date"] = feature_extract_df['sldatime'].dt.date
# last_week_df["just_date"] = last_week_df['sldatime'].dt.date
# last_month_df["just_date"] = last_month_df['sldatime'].dt.date
# global_x_axis = [1.0, 2.0, 3.0, 4.0]
# global_month_names = ["first", "second", "fourth", "fifth"]

# test
# directory = "test"
# feature_extract_df = df.copy().query('20160301 <= sldatime < 20160701')
# last_week_df = df.copy().query('20160624 <= sldatime < 20160701')
# last_month_df = df.copy().query('20160601 <= sldatime < 20160701')
# label_df = df.copy().query('20160701 <= sldatime < 20160801')
# feature_extract_df["just_date"] = feature_extract_df['sldatime'].dt.date
# last_week_df["just_date"] = last_week_df['sldatime'].dt.date
# last_month_df["just_date"] = last_month_df['sldatime'].dt.date
# global_x_axis = [1.0, 2.0, 3.0, 4.0]
# global_month_names = ["first", "second", "fourth", "fifth"]

# predict
directory = "predict"
feature_extract_df = df.copy().query('20160401 <= sldatime < 20160801')
last_week_df = df.copy().query('20160725 <= sldatime < 20160801')
last_month_df = df.copy().query('20160701 <= sldatime < 20160801')
# label_df = df.copy().query('20160801 <= sldatime < 20160901')
feature_extract_df["just_date"] = feature_extract_df['sldatime'].dt.date
last_week_df["just_date"] = last_week_df['sldatime'].dt.date
last_month_df["just_date"] = last_month_df['sldatime'].dt.date
global_x_axis = [1.0, 2.0, 3.0, 4.0]
global_month_names = ["first", "second", "fourth", "fifth"]

feature_extract_df.head()

,Unnamed: 0,uid,sldatime,pno,cno,cmrid,vipno,id,pluno,bcd,...,bndno,bndname,qty,amt,disamt,ismmx,mtype,mdocno,isdel,just_date
19869,1322202,16040109531802715,2016-04-01 09:53:45,18,8307,女[26 - 30],2900000863286,3,24011108,215087708729025980,...,-1.000000,-1,2.598000,87.290000,0.000000,0,-1,-1.000000,0,2016-04-01
6322,476579,16040109531802715,2016-04-01 09:53:45,18,8307,女[26 - 30],2900000863286,2,14845009,6936749501109,...,14104.000000,好侍,1.000000,10.600000,0.000000,0,-1,-1.000000,0,2016-04-01
6321,476578,16040109531802715,2016-04-01 09:53:45,18,8307,女[26 - 30],2900000863286,1,14845009,6936749501109,...,14104.000000,好侍,1.000000,10.600000,0.000000,0,-1,-1.000000,0,2016-04-01
20555,1365213,16040110071802720,2016-04-01 10:07:00,18,8307,女[26 - 30],2900001437424,1,14525018,6901447007406,...,14650.000000,蓝岸,1.000000,16.990000,0.000000,1,快讯促销,521866151357516.000000,0,2016-04-01
10456,711991,16040110071802720,2016-04-01 10:07:00,18,8307,女[26 - 30],2900001437424,2,15605013,6923644267070,...,15052.000000,蒙牛,1.000000,4.900000,0.000000,0,-1,-1.000000,0,2016-04-01


In [3]:
def getMonthlyNames(prefix, suffix):
    global global_month_names
    month_names = global_month_names
    res_list = []
    for n in month_names:
        res_list.append(prefix + "_" + n + "_" + suffix)
    return res_list
def getTotalCounts(i_df, group_by, use_col, new_col_name, isUnique):
    input_df = i_df.copy()
    res_df = None
    if isUnique:
        res_df = input_df.groupby(group_by)[use_col].nunique().reset_index().fillna(0)
    else:
        res_df = input_df.groupby(group_by)[use_col].count().reset_index().fillna(0)
    res_df.columns = group_by + [new_col_name]
    return res_df

def getMonthlyCounts(i_df, group_by, use_col, new_cols_name, isUnique):
    input_df = i_df.copy()
    input_df.index = input_df["sldatime"]
    res_df = None
    if isUnique:
        res_df = input_df.groupby(group_by + [input_df.index.month])[use_col].nunique().unstack().reset_index().fillna(0)
    else:
        res_df = input_df.groupby(group_by + [input_df.index.month])[use_col].count().unstack().reset_index().fillna(0)
    res_df = res_df.rename_axis(None, axis=1)
    res_df.columns = group_by + new_cols_name
#     print(group_by + new_cols_name)
    return res_df

def getTotalSum(i_df, group_by, use_col, new_col_name):
    input_df = i_df.copy()
    res_df = None
    res_df = input_df.groupby(group_by)[use_col].sum().reset_index().fillna(0)
    res_df.columns = group_by + [new_col_name]
    return res_df

def getMonthlySum(i_df, group_by, use_col, new_cols_name):
    input_df = i_df.copy()
    input_df.index = input_df["sldatime"]
    res_df = None
    res_df = input_df.groupby(group_by + [input_df.index.month])[use_col].sum().unstack().reset_index().fillna(0)
    res_df = res_df.rename_axis(None, axis=1)
    res_df.columns = group_by + new_cols_name
    return res_df

def getMonthAgg(monthly_df, index_cols, start, basic_name):
    input_df = monthly_df.copy()
    res_df = monthly_df.copy()
    mean_name = basic_name + "_mean"
    std_name = basic_name + "_std"
    max_name = basic_name + "_max"
    median_name = basic_name + "_median"
    res_df[mean_name] = input_df.iloc[:,start:].mean(axis=1)
    res_df[std_name] = input_df.iloc[:,start:].std(axis=1)
    res_df[max_name] = input_df.iloc[:,start:].max(axis=1)
    res_df[median_name] = input_df.iloc[:,start:].median(axis=1)
    res_df = res_df[index_cols + [mean_name, std_name, max_name, median_name]]
    return res_df

def getUIBCAgg(input_df, group_by, save_col, use_col, start, basic_name, agg_type):
    res_df = None
    if agg_type == 0:
        input_df = input_df.copy().groupby(group_by)[use_col].count().unstack().reset_index().rename_axis(None, axis=1)
    elif agg_type == 1:
        input_df = input_df.copy().groupby(group_by)[use_col].nunique().unstack().reset_index().rename_axis(None, axis=1)
    elif agg_type == 2:
        input_df = input_df.copy().groupby(group_by)[use_col].sum().unstack().reset_index().rename_axis(None, axis=1)
    res_df = input_df.copy()
    mean_name = basic_name + "_mean"
    std_name = basic_name + "_std"
    max_name = basic_name + "_max"
    median_name = basic_name + "_median"
    res_df[mean_name] = input_df.iloc[:,start:].mean(axis=1).fillna(0)
    res_df[std_name] = input_df.iloc[:,start:].std(axis=1).fillna(0)
    res_df[max_name] = input_df.iloc[:,start:].max(axis=1).fillna(0)
    res_df[median_name] = input_df.iloc[:,start:].median(axis=1).fillna(0)
    res_df = res_df[[save_col, mean_name, std_name, max_name, median_name]]
    return res_df

def getRepeatFeature(input_df, group_by, basic_name):
    res_df = None
    i_df = input_df.copy()
    temp = getTotalCounts(i_df, group_by, "just_date", "temp_counts", True)
    temp2 = getTotalCounts(feature_extract_df, [group_by[0]], group_by[1], "temp2_total_buyer_counts", True)
    temp = temp.loc[temp["temp_counts"] > 1]
    temp = getTotalCounts(temp, [group_by[0]], group_by[1], basic_name + "_counts", False)
    res_df = temp2.merge(temp, how="left").fillna(0)
    res_df[basic_name+"_ratio"] = res_df[basic_name + "_counts"] / res_df["temp2_total_buyer_counts"]
    res_df = res_df[[group_by[0], basic_name+"_counts", basic_name+"_ratio"]]
    return res_df

def commonMergeDataFrame(input_dict, other_list = []):
    res_df = None
    for key, value in input_dict.items():
        if res_df is None:
            res_df = value
            continue
        if key not in other_list:
            res_df = res_df.merge(value)
        else:
            res_df = res_df.merge(value, how="left").fillna(0)
    return res_df

def getTrend(x):
    global global_x_axis
    a = global_x_axis
    b = x[-len(global_x_axis):]
    slope, intercept, r_value, p_value, std_err = stats.linregress(a,b)
    return slope
                           
def mergeProfileFrame(input_df_list):
    res_df = None
    for i in input_df_list:
        if res_df is None:
            res_df = i.copy()
            continue
        res_df = res_df.merge(i.copy())
    return res_df
def mergeFeatureWithLabel(feature_df, label_df, on_cols):
    res_df = None
    f_df = feature_df.copy()
    l_df = label_df.copy()[on_cols]
    l_df = l_df.drop_duplicates()
    l_df["label"] = "1"
    res_df = f_df.merge(l_df, how="left").fillna({"label": "0"})
    res_df[["label"]] = res_df[["label"]].astype(int)
    return res_df
def isDifferent(raw_df, groupby, new_df):
    s1 = raw_df[groupby].drop_duplicates().shape
    s2 = new_df.shape
    print(s1)
    print(s2)
    return s1[0] == s2[0]
def getDeviation(raw_df, agg_df, index_cols, new_names, basic_name):
    res_df = None
    r_df = raw_df.copy()
    a_df = agg_df.copy()
    res_df = raw_df.merge(a_df)
    res_df["few_months_ago_mean"] = res_df.loc[:,new_names[0]:new_names[-2]].mean(axis=1)
    res_df["few_months_ago_std"] = res_df.loc[:,new_names[0]:new_names[-2]].std(axis=1)
    res_df[basic_name+"_bias"] = res_df[new_names[-1]] - res_df["few_months_ago_mean"]
    res_df[basic_name+"_bias_divide_mean"] = res_df[basic_name+"_bias"] / res_df["few_months_ago_mean"]
    res_df[basic_name+"_bias_divide_std"] = res_df[basic_name+"_bias"] / res_df["few_months_ago_std"]
    res_df = res_df[index_cols+[basic_name+"_bias", basic_name+"_bias_divide_std", basic_name+"_bias_divide_mean"]]
    where_are_nan = np.isnan(res_df)  
    where_are_inf = np.isinf(res_df)  
    res_df[where_are_nan] = 0  
    res_df[where_are_inf] = 0 
    return res_df.fillna(0)

### user profile

In [4]:
# 463个用户
user_profile_dict = {}

# 
# TYPE.1 count/ratio - count
# whold period
# 1. 用户购物总次数
# 2. 用户购物花钱总金额
# 3. Day Count - 用户购物总天数（频率）
#
user_profile_dict["up_plu_counts_df"] = getTotalCounts(feature_extract_df, ["vipno"], "pluno", "up_plu_counts", False)
user_profile_dict["up_plu_sum_df"] = getTotalSum(feature_extract_df, ["vipno"], "amt", "up_plu_sum")
user_profile_dict["up_plu_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno"], "just_date", "up_plu_day_ucounts", True)

# 
# TYPE.1 count/ratio - count
# monthly
# 1. 用户购物月次数
# 2. 用户购物花钱月金额
# 3. 用户购物月天数（月频率）
plu_monthly_counts_new_names = getMonthlyNames("up", "plu_counts")
user_profile_dict["up_monthly_plu_counts_df"] = getMonthlyCounts(feature_extract_df, ["vipno"], "pluno", plu_monthly_counts_new_names, False)
plu_sum_new_names = getMonthlyNames("up", "plu_sum")
user_profile_dict["up_monthly_plu_sum_df"] = getMonthlySum(feature_extract_df, ["vipno"], "amt", plu_sum_new_names)
plu_day_ucounts_new_names = getMonthlyNames("up", "plu_day_ucounts")
user_profile_dict["up_monthly_plu_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno"], "pluno", plu_day_ucounts_new_names, False)

#
# TYPE.1 count/ratio - product diversity
# whold period
# 1. 购买的unique的I的总数量 -- (用户买过几种商品，去重)
# 2. 购买的unique的B的总数量 -- (用户买过几种品牌，去重)
# 3. 购买的unique的C的总数量 -- (用户买过几种类型，去重)
user_profile_dict["up_plu_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno"], "pluno", "up_plu_ucounts", True)
user_profile_dict["up_bnd_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno"], "bndno", "up_bnd_ucounts", True)
user_profile_dict["up_dpt_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno"], "dptno", "up_dpt_ucounts", True)

# TYPE.1 count/ratio - product diversity
# monthly
# 1. 购买的unique的I的月数量
# 2. 购买的unique的B的月数量
# 3. 购买的unique的C的月数量
plu_ucounts_new_names = getMonthlyNames("up", "plu_ucounts")
bnd_ucounts_new_names = getMonthlyNames("up", "bnd_ucount")
dpt_ucounts_new_names = getMonthlyNames("up", "dpt_ucounts")
user_profile_dict["up_monthly_plu_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno"], "pluno", plu_ucounts_new_names, True)
user_profile_dict["up_monthly_bnd_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno"], "bndno", bnd_ucounts_new_names, True)
user_profile_dict["up_monthly_dpt_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno"], "dptno", dpt_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
user_profile_dict["up_monthly_plu_counts_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_plu_counts_df"], ["vipno"], 1, "up_monthly_plu_counts")
user_profile_dict["up_monthly_plu_sum_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_plu_sum_df"], ["vipno"], 1, "up_monthly_plu_sum")
user_profile_dict["up_monthly_plu_day_ucounts_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_plu_day_ucounts_df"], ["vipno"], 1, "up_monthly_plu_day_ucounts")
# 2. agg: TYPE.1 count/ratio - product diversity
user_profile_dict["up_monthly_plu_ucounts_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_plu_ucounts_df"], ["vipno"], 1, "up_monthly_plu_ucounts")
user_profile_dict["up_monthly_bnd_ucounts_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_bnd_ucounts_df"], ["vipno"], 1, "up_monthly_bnd_ucounts")
user_profile_dict["up_monthly_dpt_ucounts_agg_df"] = getMonthAgg(user_profile_dict["up_monthly_dpt_ucounts_df"], ["vipno"], 1, "up_monthly_dpt_ucounts")

#
# TYPE.2 AGG feature - brand/category/item AGG 衡量用户的购买习惯
# 每组内先针对单个B、C、I进行统计，再进行aggregation 针对B、C、I统计
# 1. 总时间内发生被购买的天数
# 2. 总时间内被购买的次数
# 3. 总时间被购买的金额getUIBCAgg(feature_extract_df, ["pluno", "vipno"], "pluno", "id", 2, "pp_plu_user_agg_counts", 0)
# agg操作包含mean、std、max、median pp_plu_user_agg_counts_df
# plu
user_profile_dict["up_u_plu_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "pluno"], "vipno", "id", 2, "up_u_plu_agg_counts", 0)
user_profile_dict["up_u_plu_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["vipno", "pluno"], "vipno", "amt", 2, "up_u_plu_agg_sum", 2)
user_profile_dict["up_u_plu_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "pluno"], "vipno", "just_date", 2, "up_u_plu_agg_day_ucounts", 1)
# bnd
user_profile_dict["up_u_bnd_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "bndno"], "vipno", "id", 2, "up_u_bnd_agg_counts", 0)
user_profile_dict["up_u_bnd_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["vipno", "bndno"], "vipno", "amt", 2, "up_u_bnd_agg_sum", 2)
user_profile_dict["up_u_bnd_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "bndno"], "vipno", "just_date", 2, "up_u_bnd_agg_day_ucounts", 1)
# dpt
user_profile_dict["up_u_dpt_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "dptno"], "vipno", "id", 2, "up_u_dpt_agg_counts", 0)
user_profile_dict["up_u_dpt_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["vipno", "dptno"], "vipno", "amt", 2, "up_u_dpt_agg_sum", 2)
user_profile_dict["up_u_dpt_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["vipno", "dptno"], "vipno", "just_date", 2, "up_u_dpt_agg_day_ucounts", 1)

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week（过去7天）
# TYPE.1 count/ratio - count
user_profile_dict["up_l_w_plu_counts_df"] = getTotalCounts(last_week_df, ["vipno"], "pluno", "up_l_w_plu_counts", False)
user_profile_dict["up_l_w_plu_sum_df"] = getTotalSum(last_week_df, ["vipno"], "amt", "up_l_w_plu_sum")
user_profile_dict["up_l_w_plu_day_ucounts_df"] = getTotalCounts(last_week_df, ["vipno"], "just_date", "up_l_w_plu_day_ucounts", True)
# last week（过去7天）
# TYPE.1 count/ratio - product diversity
user_profile_dict["up_l_w_plu_ucounts_df"] = getTotalCounts(last_week_df, ["vipno"], "pluno", "up_l_w_plu_ucounts", True)
user_profile_dict["up_l_w_bnd_ucounts_df"] = getTotalCounts(last_week_df, ["vipno"], "bndno", "up_l_w_bnd_ucounts", True)
user_profile_dict["up_l_w_dpt_ucounts_df"] = getTotalCounts(last_week_df, ["vipno"], "dptno", "up_l_w_dpt_ucounts", True)
# last month 过去一个月
# TYPE.1 count/ratio - count
user_profile_dict["up_l_m_plu_counts_df"] = getTotalCounts(last_month_df, ["vipno"], "pluno", "up_l_m_plu_counts", False)
user_profile_dict["up_l_m_plu_sum_df"] = getTotalSum(last_month_df, ["vipno"], "amt", "up_l_m_plu_sum")
user_profile_dict["up_l_m_plu_day_ucounts_df"] = getTotalCounts(last_month_df, ["vipno"], "just_date", "up_l_m_plu_day_ucounts", True)
# last month 过去一个月
# TYPE.1 count/ratio - product diversity
user_profile_dict["up_l_m_plu_ucounts_df"] = getTotalCounts(last_month_df, ["vipno"], "pluno", "up_l_m_plu_ucounts", True)
user_profile_dict["up_l_m_bnd_ucounts_df"] = getTotalCounts(last_month_df, ["vipno"], "bndno", "up_l_m_bnd_ucounts", True)
user_profile_dict["up_l_m_dpt_ucounts_df"] = getTotalCounts(last_month_df, ["vipno"], "dptno", "up_l_m_dpt_ucounts", True)

# 
# TYPE.4 complex feature - repeat feature
#统计repeat {I, B, C}的count/ratio
#1. count:被该用户重复购买过的商品的数量
#2. ratio:count / 该用户购买过的商品总数
user_profile_dict["up_repeat_plu_df"] = getRepeatFeature(feature_extract_df, ["vipno", "pluno"], "up_repeat_plu")
user_profile_dict["up_repeat_bnd_df"] = getRepeatFeature(feature_extract_df, ["vipno", "bndno"], "up_repeat_bnd")
user_profile_dict["up_repeat_dpt_df"] = getRepeatFeature(feature_extract_df, ["vipno", "dptno"], "up_repeat_dpt")

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
user_profile_dict["up_monthly_plu_counts_df"]["up_trend_plu_counts"] = user_profile_dict["up_monthly_plu_counts_df"].apply(getTrend, axis=1)
user_profile_dict["up_monthly_plu_sum_df"]["up_trend_plu_sum"] = user_profile_dict["up_monthly_plu_sum_df"].apply(getTrend, axis=1)
user_profile_dict["up_monthly_plu_day_ucounts_df"]["up_trend_plu_day_ucounts"] = user_profile_dict["up_monthly_plu_day_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - product diversity
user_profile_dict["up_monthly_plu_ucounts_df"]["up_trend_plu_ucounts"] = user_profile_dict["up_monthly_plu_ucounts_df"].apply(getTrend, axis=1)
user_profile_dict["up_monthly_bnd_ucounts_df"]["up_trend_bnd_ucounts"] = user_profile_dict["up_monthly_bnd_ucounts_df"].apply(getTrend, axis=1)
user_profile_dict["up_monthly_dpt_ucounts_df"]["up_trend_dpt_ucounts"] = user_profile_dict["up_monthly_dpt_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
user_profile_dict["up_bias_ratio_plu_count_df"] = getDeviation(user_profile_dict["up_monthly_plu_counts_df"], user_profile_dict["up_monthly_plu_counts_agg_df"], ["vipno"], plu_monthly_counts_new_names, "up_plu_counts")
user_profile_dict["up_bias_ratio_plu_sum_df"] = getDeviation(user_profile_dict["up_monthly_plu_sum_df"], user_profile_dict["up_monthly_plu_sum_agg_df"], ["vipno"], plu_sum_new_names, "up_plu_sum")
user_profile_dict["up_bias_ratio_plu_day_ucounts_df"] = getDeviation(user_profile_dict["up_monthly_plu_day_ucounts_df"], user_profile_dict["up_monthly_plu_day_ucounts_agg_df"], ["vipno"], plu_day_ucounts_new_names, "up_plu_day_ucounts")
# TYPE.1 count/ratio - product diversity
user_profile_dict["up_bias_ratio_plu_ucounts_df"] =  getDeviation(user_profile_dict["up_monthly_plu_ucounts_df"], user_profile_dict["up_monthly_plu_ucounts_agg_df"], ["vipno"], plu_ucounts_new_names, "up_plu_ucounts")
user_profile_dict["up_bias_ratio_bnd_ucounts_df"] =  getDeviation(user_profile_dict["up_monthly_bnd_ucounts_df"], user_profile_dict["up_monthly_bnd_ucounts_agg_df"], ["vipno"], bnd_ucounts_new_names, "up_bnd_ucounts")
user_profile_dict["up_bias_ratio_dpt_ucounts_df"] =  getDeviation(user_profile_dict["up_monthly_dpt_ucounts_df"], user_profile_dict["up_monthly_dpt_ucounts_agg_df"], ["vipno"], dpt_ucounts_new_names, "up_dpt_ucounts")

other_list = ["up_l_w_plu_counts_df", "up_l_w_plu_sum_df", "up_l_w_plu_day_ucounts_df",
             "up_l_w_plu_ucounts_df", "up_l_w_bnd_ucounts_df", "up_l_w_dpt_ucounts_df",
             "up_l_m_plu_counts_df", "up_l_m_plu_sum_df", "up_l_m_plu_day_ucounts_df",
             "up_l_m_plu_ucounts_df", "up_l_m_bnd_ucounts_df", "up_l_m_dpt_ucounts_df"]
user_profile_df = commonMergeDataFrame(user_profile_dict, other_list)
assert isDifferent(feature_extract_df, ["vipno"], user_profile_df)
user_profile_df.sort_values("vipno").head()

# user_profile_df.to_csv("./csv/user_profile.csv")
user_profile_dict["up_plu_counts_df"].shape

(451, 1)
(451, 133)


(451, 2)

### user-plu profile

In [5]:
# 13794个UP对
u_p_profile_dict = {}

#
# TYPE.1 count/ratio - count
# whold period
# 1. U-P对总次数 使用id表示订单内重复购买某个商品也算是一次
# 2. U-P对总金额
# 3. U-P对总出现天数， 这里基本上和U-P对总次数差不多，因为同一天只有一个订单，并且一个订单同样的商品只有一个
u_p_profile_dict["upp_up_counts_df"] = getTotalCounts(feature_extract_df, ["vipno","pluno"], "id", "upp_up_counts", False)
u_p_profile_dict["upp_up_sum_df"] = getTotalSum(feature_extract_df, ["vipno","pluno"], "amt", "upp_up_sum")
u_p_profile_dict["upp_up_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno","pluno"], "just_date", "upp_up_day_ucounts", True)

#
# TYPE.1 count/ratio - count
# monthly
# 1. U-P对月次数
# 2. U-P对月金额
# 3. U-P对月出现天数
upp_monthly_up_counts_new_names = getMonthlyNames("upp", "up_counts")
upp_monthly_up_sum_new_names = getMonthlyNames("upp", "up_sum")
upp_monthly_up_day_ucounts_new_names = getMonthlyNames("upp", "up_day_ucounts")
u_p_profile_dict["upp_monthly_up_counts_df"] = getMonthlyCounts(feature_extract_df, ["vipno","pluno"], "id", upp_monthly_up_counts_new_names, False)
u_p_profile_dict["upp_monthly_up_sum_df"] = getMonthlySum(feature_extract_df, ["vipno","pluno"], "amt", upp_monthly_up_sum_new_names)
u_p_profile_dict["upp_monthly_up_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno","pluno"], "just_date", upp_monthly_up_day_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
u_p_profile_dict["upp_monthly_up_counts_agg_df"] = getMonthAgg(u_p_profile_dict["upp_monthly_up_counts_df"], ["vipno","pluno"], 2, "upp_monthly_up_counts")
u_p_profile_dict["upp_monthly_up_sum_agg_df"] = getMonthAgg(u_p_profile_dict["upp_monthly_up_sum_df"], ["vipno","pluno"], 2, "upp_monthly_up_sum")
u_p_profile_dict["upp_monthly_up_day_ucounts_agg_df"] = getMonthAgg(u_p_profile_dict["upp_monthly_up_day_ucounts_df"], ["vipno","pluno"], 2, "upp_monthly_up_day_ucounts")

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week（过去7天）
# TYPE.1 count/ratio - count
u_p_profile_dict["upp_up_l_w_counts_df"] = getTotalCounts(last_week_df, ["vipno","pluno"], "id", "upp_up_l_w_counts", False)
u_p_profile_dict["upp_up_l_w_sum_df"] = getTotalSum(last_week_df, ["vipno","pluno"], "amt", "upp_up_l_w_sum")
u_p_profile_dict["upp_up_l_w_day_ucounts_df"] = getTotalCounts(last_week_df, ["vipno","pluno"], "just_date", "upp_up_l_w_day_ucounts", True)
# last month
# TYPE.1 count/ratio - count
u_p_profile_dict["upp_up_l_m_counts_df"] = getTotalCounts(last_month_df, ["vipno","pluno"], "id", "upp_up_l_m_counts", False)
u_p_profile_dict["upp_up_l_m_sum_df"] = getTotalSum(last_month_df, ["vipno","pluno"], "amt", "upp_up_l_m_sum")
u_p_profile_dict["upp_up_l_m_day_ucounts_df"] = getTotalCounts(last_month_df, ["vipno","pluno"], "just_date", "upp_up_l_m_day_ucounts", True)


#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
u_p_profile_dict["upp_monthly_up_counts_df"]["upp_trend_up_counts"] = u_p_profile_dict["upp_monthly_up_counts_df"].apply(getTrend, axis=1)
u_p_profile_dict["upp_monthly_up_sum_df"]["upp_trend_up_sum"] = u_p_profile_dict["upp_monthly_up_sum_df"].apply(getTrend, axis=1)
u_p_profile_dict["upp_monthly_up_day_ucounts_df"]["upp_trend_up_day_ucounts"] =u_p_profile_dict["upp_monthly_up_day_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
u_p_profile_dict["upp_bias_ratio_up_counts_df"] = getDeviation(u_p_profile_dict["upp_monthly_up_counts_df"], u_p_profile_dict["upp_monthly_up_counts_agg_df"], ["vipno","pluno"], upp_monthly_up_counts_new_names, "upp_up_counts")
u_p_profile_dict["upp_bias_ratio_up_sum_df"] = getDeviation(u_p_profile_dict["upp_monthly_up_sum_df"], u_p_profile_dict["upp_monthly_up_sum_agg_df"], ["vipno","pluno"], upp_monthly_up_sum_new_names, "upp_up_sum")
u_p_profile_dict["upp_bias_ratio_up_day_ucounts_df"] = getDeviation(u_p_profile_dict["upp_monthly_up_day_ucounts_df"], u_p_profile_dict["upp_monthly_up_day_ucounts_agg_df"], ["vipno","pluno"], upp_monthly_up_day_ucounts_new_names, "upp_up_day_ucounts")

upp_other_list = ["upp_up_l_w_counts_df", "upp_up_l_w_sum_df", "upp_up_l_w_day_ucounts_df",
                 "upp_up_l_m_counts_df", "upp_up_l_m_sum_df", "upp_up_l_m_day_ucounts_df"]
u_p_profile_df = commonMergeDataFrame(u_p_profile_dict, upp_other_list)
assert isDifferent(feature_extract_df, ["vipno","pluno"], u_p_profile_df)
u_p_profile_df.sort_values("vipno").head()

# user_profile_df.to_csv("./csv/u_p_profile.csv")

(10981, 2)
(10981, 47)


,vipno,pluno,upp_up_counts,upp_up_sum,upp_up_day_ucounts,upp_first_up_counts,upp_second_up_counts,upp_fourth_up_counts,upp_fifth_up_counts,upp_trend_up_counts,...,upp_up_l_m_day_ucounts,upp_up_counts_bias,upp_up_counts_bias_divide_std,upp_up_counts_bias_divide_mean,upp_up_sum_bias,upp_up_sum_bias_divide_std,upp_up_sum_bias_divide_mean,upp_up_day_ucounts_bias,upp_up_day_ucounts_bias_divide_std,upp_up_day_ucounts_bias_divide_mean
0,781924,10113009,1,6.800000,1,0.000000,1.000000,0.000000,0.000000,-0.100000,...,0.000000,-0.333333,-0.577350,-1.000000,-2.266667,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
19,781924,34023002,1,15.000000,1,0.000000,0.000000,0.000000,1.000000,0.300000,...,1.000000,1.000000,0.000000,0.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000
18,781924,23113024,1,18.000000,1,0.000000,0.000000,0.000000,1.000000,0.300000,...,1.000000,1.000000,0.000000,0.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000
17,781924,15202012,1,4.900000,1,0.000000,0.000000,1.000000,0.000000,0.100000,...,0.000000,-0.333333,-0.577350,-1.000000,-1.633333,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
16,781924,15200007,3,101.200000,3,1.000000,1.000000,1.000000,0.000000,-0.300000,...,0.000000,-1.000000,0.000000,-1.000000,-33.733333,-6.350853,-1.000000,-1.000000,0.000000,-1.000000


### user-bnd profile -1代表三无产品

In [6]:
# 6446个UB对
u_b_profile_dict = {}

# 
# TYPE.1 count/ratio - count
# whold period
# 1. U-B对总次数 使用id表示订单内重复购买某个商品也算是一次
# 2. U-B总金额
# 3. U-B对总出现天数, 与UP不同的是，虽然同一天只有一个订单，同一个订单里相同的商品只有一项，但相同品牌却容易出现多项
u_b_profile_dict["ubp_ub_counts_df"] = getTotalCounts(feature_extract_df, ["vipno","bndno"], "id", "ubp_ub_counts", False)
u_b_profile_dict["ubp_ub_sum_df"] = getTotalSum(feature_extract_df, ["vipno","bndno"], "amt", "ubp_ub_sum")
u_b_profile_dict["ubp_ub_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno","bndno"], "just_date", "ubp_ub_day_ucounts", True)

# 
# TYPE.1 count/ratio - count
# monthly
# 1. 购买(主体为用户)/被购买(主体为B、C、I)的次数
# 2. 购买(主体为用户)/被购买(主体为B、C、I)的金额
# 3. 购买/被购买发生的天数
ubp_monthly_ub_counts_new_names = getMonthlyNames("ubp", "ub_counts")
ubp_monthly_ub_sum_new_names = getMonthlyNames("ubp", "ub_sum")
ubp_monthly_ub_day_ucounts_new_names = getMonthlyNames("ubp", "ub_day_ucounts")
u_b_profile_dict["ubp_monthly_ub_counts_df"] = getMonthlyCounts(feature_extract_df, ["vipno","bndno"], "id", ubp_monthly_ub_counts_new_names, False)
u_b_profile_dict["ubp_monthly_ub_sum_df"] = getMonthlySum(feature_extract_df, ["vipno","bndno"], "amt", ubp_monthly_ub_sum_new_names)
u_b_profile_dict["ubp_monthly_ub_day_ucounts_df"] =  getMonthlyCounts(feature_extract_df, ["vipno","bndno"], "just_date", ubp_monthly_ub_day_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
u_b_profile_dict["ubp_monthly_ub_counts_agg_df"] = getMonthAgg(u_b_profile_dict["ubp_monthly_ub_counts_df"], ["vipno","bndno"], 2, "ubp_monthly_ub_counts")
u_b_profile_dict["ubp_monthly_ub_sum_agg_df"] = getMonthAgg(u_b_profile_dict["ubp_monthly_ub_sum_df"], ["vipno","bndno"], 2, "ubp_monthly_ub_sum")
u_b_profile_dict["ubp_monthly_ub_day_ucounts_agg_df"]  = getMonthAgg(u_b_profile_dict["ubp_monthly_ub_day_ucounts_df"], ["vipno","bndno"], 2, "ubp_monthly_ub_day_ucounts")

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week
# TYPE.1 count/ratio - count
u_b_profile_dict["ubp_ub_l_w_counts_df"] = getTotalCounts(last_week_df, ["vipno","bndno"], "id", "ubp_ub_l_w_counts", False)
u_b_profile_dict["ubp_ub_l_w_sum_df"] = getTotalSum(last_week_df, ["vipno","bndno"], "amt", "ubp_ub_l_w_sum")
u_b_profile_dict["ubp_ub_l_w_day_ucounts_df"] = getTotalCounts(last_week_df, ["vipno","bndno"], "just_date", "ubp_ub_l_w_day_ucounts", True)
# last month
# TYPE.1 count/ratio - count
u_b_profile_dict["ubp_ub_l_m_counts_df"] = getTotalCounts(last_month_df, ["vipno","bndno"], "id", "ubp_ub_l_m_counts", False)
u_b_profile_dict["ubp_ub_l_m_sum_df"] = getTotalSum(last_month_df, ["vipno","bndno"], "amt", "ubp_ub_l_m_sum")
u_b_profile_dict["ubp_ub_l_m_day_ucounts_df"] = getTotalCounts(last_month_df, ["vipno","bndno"], "just_date", "ubp_ub_l_m_day_ucounts", True)

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
u_b_profile_dict["ubp_monthly_ub_counts_df"]["ubp_trend_ub_counts_df"] = u_b_profile_dict["ubp_monthly_ub_counts_df"].apply(getTrend, axis=1)
u_b_profile_dict["ubp_monthly_ub_sum_df"]["ubp_trend_ub_sum_df"] = u_b_profile_dict["ubp_monthly_ub_sum_df"].apply(getTrend, axis=1)
u_b_profile_dict["ubp_monthly_ub_day_ucounts_df"]["ubp_trend_ub_day_ucounts_df"] =  u_b_profile_dict["ubp_monthly_ub_day_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
u_b_profile_dict["ubp_bias_ratio_ub_counts_df"] = getDeviation(u_b_profile_dict["ubp_monthly_ub_counts_df"],u_b_profile_dict["ubp_monthly_ub_counts_agg_df"], ["vipno","bndno"], ubp_monthly_ub_counts_new_names, "ubp_ub_counts")
u_b_profile_dict["ubp_bias_ratio_ub_sum_df"] = getDeviation(u_b_profile_dict["ubp_monthly_ub_sum_df"], u_b_profile_dict["ubp_monthly_ub_sum_agg_df"], ["vipno","bndno"], ubp_monthly_ub_sum_new_names, "ubp_ub_sum")
u_b_profile_dict["ubp_bias_ratio_ub_day_ucounts_df"] = getDeviation(u_b_profile_dict["ubp_monthly_ub_day_ucounts_df"], u_b_profile_dict["ubp_monthly_ub_day_ucounts_agg_df"], ["vipno","bndno"], ubp_monthly_ub_day_ucounts_new_names, "ubp_ub_day_ucounts")


ubp_other_list = ["ubp_ub_l_w_counts_df", "ubp_ub_l_w_sum_df", "ubp_ub_l_w_day_ucounts_df", 
                 "ubp_ub_l_m_counts_df", "ubp_ub_l_m_sum_df", "ubp_ub_l_m_day_ucounts_df"]
u_b_profile_df = commonMergeDataFrame(u_b_profile_dict, ubp_other_list)
assert isDifferent(feature_extract_df, ["vipno","bndno"], u_b_profile_df)
u_b_profile_df.head()


(5339, 2)
(5339, 47)


,vipno,bndno,ubp_ub_counts,ubp_ub_sum,ubp_ub_day_ucounts,ubp_first_ub_counts,ubp_second_ub_counts,ubp_fourth_ub_counts,ubp_fifth_ub_counts,ubp_trend_ub_counts_df,...,ubp_ub_l_m_day_ucounts,ubp_ub_counts_bias,ubp_ub_counts_bias_divide_std,ubp_ub_counts_bias_divide_mean,ubp_ub_sum_bias,ubp_ub_sum_bias_divide_std,ubp_ub_sum_bias_divide_mean,ubp_ub_day_ucounts_bias,ubp_ub_day_ucounts_bias_divide_std,ubp_ub_day_ucounts_bias_divide_mean
0,781924,-1.000000,5,73.720000,3,1.000000,3.000000,0.000000,1.000000,-0.300000,...,1.000000,-0.333333,-0.218218,-0.250000,-0.573333,-0.024452,-0.030869,0.333333,0.577350,0.500000
1,781924,10106.000000,1,5.500000,1,1.000000,0.000000,0.000000,0.000000,-0.300000,...,0.000000,-0.333333,-0.577350,-1.000000,-1.833333,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
2,781924,10706.000000,1,6.800000,1,0.000000,1.000000,0.000000,0.000000,-0.100000,...,0.000000,-0.333333,-0.577350,-1.000000,-2.266667,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
3,781924,11129.000000,1,9.900000,1,1.000000,0.000000,0.000000,0.000000,-0.300000,...,0.000000,-0.333333,-0.577350,-1.000000,-3.300000,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
4,781924,11149.000000,1,15.600000,1,1.000000,0.000000,0.000000,0.000000,-0.300000,...,0.000000,-0.333333,-0.577350,-1.000000,-5.200000,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000


### user-dpt profile

In [7]:
# 10845个UD对
u_d_profile_dict = {}

# 
# TYPE.1 count/ratio - count
# whold period
# 1. U-D对总次数
# 2. U-D对总金额
# 3. U-D对总出现天数
u_d_profile_dict["udp_ud_counts_df"] = getTotalCounts(feature_extract_df, ["vipno","dptno"], "id", "udp_ud_counts", False)
u_d_profile_dict["udp_ud_sum_df"] = getTotalSum(feature_extract_df, ["vipno","dptno"], "amt", "udp_ud_sum")
u_d_profile_dict["udp_ud_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["vipno","dptno"], "just_date", "udp_ud_day_ucounts", True)

# 
# TYPE.1 count/ratio - count
# Monthly
# 1. U-D对月次数
# 2. U-D对月金额
# 3. U-D对月出现天数
udp_monthly_ud_counts_new_names = getMonthlyNames("udp", "ud_counts")
udp_monthly_ud_sum_new_names = getMonthlyNames("udp", "ud_sum")
udp_monthly_ud_day_ucounts_new_names = getMonthlyNames("udp", "ud_day_ucounts")
u_d_profile_dict["udp_monthly_ud_counts_df"] = getMonthlyCounts(feature_extract_df, ["vipno","dptno"], "id", udp_monthly_ud_counts_new_names, False)
u_d_profile_dict["udp_monthly_ud_sum_df"] = getMonthlySum(feature_extract_df, ["vipno","dptno"], "amt", udp_monthly_ud_sum_new_names)
u_d_profile_dict["udp_monthly_ud_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["vipno","dptno"], "just_date", udp_monthly_ud_day_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
u_d_profile_dict["udp_monthly_ud_counts_agg_df"] = getMonthAgg(u_d_profile_dict["udp_monthly_ud_counts_df"], ["vipno","dptno"], 2, "udp_monthly_ud_counts")
u_d_profile_dict["udp_monthly_ud_sum_agg_df"] = getMonthAgg(u_d_profile_dict["udp_monthly_ud_sum_df"], ["vipno","dptno"], 2, "udp_monthly_ud_sum")
u_d_profile_dict["udp_monthly_ud_day_ucounts_agg_df"] = getMonthAgg(u_d_profile_dict["udp_monthly_ud_day_ucounts_df"], ["vipno","dptno"], 2, "udp_monthly_ud_day_ucounts")

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week
# TYPE.1 count/ratio - count
u_d_profile_dict["udp_ud_l_w_counts_df"] = getTotalCounts(last_week_df, ["vipno","dptno"], "id", "udp_ud_l_w_counts", False)
u_d_profile_dict["udp_ud_l_w_sum_df"] = getTotalSum(last_week_df, ["vipno","dptno"], "amt", "udp_ud_l_w_sum")
u_d_profile_dict["udp_ud_l_w_day_ucounts_df"] = getTotalCounts(last_week_df, ["vipno","dptno"], "just_date", "udp_ud_l_w_day_ucounts", True)
# last month
# TYPE.1 count/ratio - count
u_d_profile_dict["udp_ud_l_m_counts_df"] = getTotalCounts(last_month_df, ["vipno","dptno"], "id", "udp_ud_l_m_counts", False)
u_d_profile_dict["udp_ud_l_m_sum_df"] = getTotalSum(last_month_df, ["vipno","dptno"], "amt", "udp_ud_l_m_sum")
u_d_profile_dict["udp_ud_l_m_day_ucounts_df"] = getTotalCounts(last_month_df, ["vipno","dptno"], "just_date", "udp_ud_l_m_day_ucounts", True)

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
u_d_profile_dict["udp_monthly_ud_counts_df"]["udp_trend_ud_counts_df"] = u_d_profile_dict["udp_monthly_ud_counts_df"].apply(getTrend, axis=1)
u_d_profile_dict["udp_monthly_ud_sum_df"]["udp_trend_ud_sum_df"] = u_d_profile_dict["udp_monthly_ud_sum_df"].apply(getTrend, axis=1)
u_d_profile_dict["udp_monthly_ud_day_ucounts_df"]["udp_trend_ud_day_ucounts_df"] = u_d_profile_dict["udp_monthly_ud_day_ucounts_df"].apply(getTrend, axis=1)


#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
u_d_profile_dict["udp_bias_ratio_ud_counts_df"] = getDeviation(u_d_profile_dict["udp_monthly_ud_counts_df"],u_d_profile_dict["udp_monthly_ud_counts_agg_df"], ["vipno","dptno"], udp_monthly_ud_counts_new_names, "udp_ud_counts")
u_d_profile_dict["udp_bias_ratio_ud_sum_df"] = getDeviation(u_d_profile_dict["udp_monthly_ud_sum_df"], u_d_profile_dict["udp_monthly_ud_sum_agg_df"], ["vipno","dptno"], udp_monthly_ud_sum_new_names, "udp_ud_sum")
u_d_profile_dict["udp_bias_ratio_ud_day_ucounts_df"] = getDeviation(u_d_profile_dict["udp_monthly_ud_day_ucounts_df"], u_d_profile_dict["udp_monthly_ud_day_ucounts_agg_df"], ["vipno","dptno"], udp_monthly_ud_day_ucounts_new_names, "udp_ud_day_ucounts")



udp_other_list = ["udp_ud_l_w_counts_df", "udp_ud_l_w_sum_df", "udp_ud_l_w_day_ucounts_df",
                 "udp_ud_l_m_counts_df", "udp_ud_l_m_sum_df", "udp_ud_l_m_day_ucounts_df"]
u_d_profile_df = commonMergeDataFrame(u_d_profile_dict, udp_other_list)
assert isDifferent(feature_extract_df, ["vipno","dptno"], u_d_profile_df)
u_d_profile_df.head()


(8804, 2)
(8804, 47)


,vipno,dptno,udp_ud_counts,udp_ud_sum,udp_ud_day_ucounts,udp_first_ud_counts,udp_second_ud_counts,udp_fourth_ud_counts,udp_fifth_ud_counts,udp_trend_ud_counts_df,...,udp_ud_l_m_day_ucounts,udp_ud_counts_bias,udp_ud_counts_bias_divide_std,udp_ud_counts_bias_divide_mean,udp_ud_sum_bias,udp_ud_sum_bias_divide_std,udp_ud_sum_bias_divide_mean,udp_ud_day_ucounts_bias,udp_ud_day_ucounts_bias_divide_std,udp_ud_day_ucounts_bias_divide_mean
0,781924,10113,1,6.800000,1,0.000000,1.000000,0.000000,0.000000,-0.100000,...,0.000000,-0.333333,-0.577350,-1.000000,-2.266667,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
1,781924,10130,1,5.500000,1,1.000000,0.000000,0.000000,0.000000,-0.300000,...,0.000000,-0.333333,-0.577350,-1.000000,-1.833333,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
2,781924,11302,1,10.800000,1,0.000000,0.000000,0.000000,1.000000,0.300000,...,1.000000,1.000000,0.000000,0.000000,10.800000,0.000000,0.000000,1.000000,0.000000,0.000000
3,781924,11531,1,15.600000,1,1.000000,0.000000,0.000000,0.000000,-0.300000,...,0.000000,-0.333333,-0.577350,-1.000000,-5.200000,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
4,781924,11532,2,13.400000,2,1.000000,0.000000,1.000000,0.000000,-0.200000,...,0.000000,-0.666667,-1.154701,-1.000000,-4.466667,-0.889722,-1.000000,-0.666667,-1.154701,-1.000000


### plu profile

In [8]:
# 4674个商品
plu_profile_dict = {}

#
# TYPE.1 count/ratio - count
# whold period
# 1. plu被购买总次数
# 2. plu被购买总金额
# 3. plu 被购买的总天数(比plu被购买总次数少，因为可能一个商品会在同一天被购买多次)
plu_profile_dict["pp_p_purchased_counts_df"] = getTotalCounts(feature_extract_df, ["pluno"], "id", "pp_p_purchased_counts", False)
plu_profile_dict["pp_p_purchased_sum_df"] = getTotalSum(feature_extract_df, ["pluno"], "amt", "pp_p_purchased_sum")
plu_profile_dict["pp_p_purchased_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["pluno"], "just_date", "pp_p_purchased_day_ucounts", True)

#
# TYPE.1 count/ratio - count
# monthly
# 1. plu每月被购买次数
# 2. plu每月被购买金额
# 3. plu每月被购买天数
pp_monthly_p_purchased_counts_new_names = getMonthlyNames("pp", "p_purchased_counts")
pp_monthly_p_purchased_sum_new_names = getMonthlyNames("pp", "p_purchased_sum")
pp_monthly_p_purchased_day_ucounts_new_names = getMonthlyNames("pp", "p_purchased_day_ucounts")
plu_profile_dict["pp_monthly_p_purchased_counts_df"] = getMonthlyCounts(feature_extract_df, ["pluno"], "id", pp_monthly_p_purchased_counts_new_names, False)
plu_profile_dict["pp_monthly_p_purchased_sum_df"] = getMonthlySum(feature_extract_df, ["pluno"], "amt", pp_monthly_p_purchased_sum_new_names)
plu_profile_dict["pp_monthly_p_purchased_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["pluno"], "just_date", pp_monthly_p_purchased_day_ucounts_new_names, True)

#
# TYPE.1 count/ratio - penetration 衡量品牌名声
# whold period
# 1. 买过该品牌或品类或商品的不同的买家的人数
plu_profile_dict["pp_p_has_u_ucounts_df"] = getTotalCounts(feature_extract_df, ["pluno"], "vipno", "pp_p_has_u_ucounts", True)

#
# TYPE.1 count/ratio - penetration 衡量品牌名声
# monthly
# 1. 每月买过该品牌或品类或商品的不同的买家的人数
pp_monthly_p_has_u_ucounts_new_names = getMonthlyNames("pp", "p_has_u_ucounts")

plu_profile_dict["pp_monthly_p_has_u_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["pluno"], "vipno", pp_monthly_p_has_u_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
plu_profile_dict["pp_monthly_p_purchased_counts_agg_df"] = getMonthAgg(plu_profile_dict["pp_monthly_p_purchased_counts_df"], ["pluno"], 1, "pp_monthly_p_purchased_counts")
plu_profile_dict["pp_monthly_p_purchased_sum_agg_df"] = getMonthAgg(plu_profile_dict["pp_monthly_p_purchased_sum_df"], ["pluno"], 1, "pp_monthly_p_purchased_sum")
plu_profile_dict["pp_monthly_p_purchased_day_ucounts_agg_df"] = getMonthAgg(plu_profile_dict["pp_monthly_p_purchased_day_ucounts_df"], ["pluno"], 1, "pp_monthly_p_purchased_day_ucounts")
# 2. agg: TYPE.1 count/ratio - penetration 衡量品牌名声
plu_profile_dict["pp_monthly_p_has_u_ucounts_agg_df"] = getMonthAgg(plu_profile_dict["pp_monthly_p_has_u_ucounts_df"], ["pluno"], 1, "pp_monthly_p_has_u_ucounts")

# TYPE.2 AGG feature - user AGG
# agg_type: 0 - count 1 - nunique 2 - sum
plu_profile_dict["pp_plu_user_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["pluno", "vipno"], "pluno", "id", 2, "pp_plu_user_agg_counts", 0)
plu_profile_dict["pp_plu_user_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["pluno", "vipno"], "pluno", "amt", 2, "pp_plu_user_agg_sum", 2)
plu_profile_dict["pp_plu_user_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["pluno", "vipno"], "pluno", "just_date", 2, "pp_plu_user_agg_day_ucounts", 1)

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week
# TYPE.1 count/ratio - count
plu_profile_dict["pp_p_l_w_purchased_counts_df"] = getTotalCounts(last_week_df, ["pluno"], "id", "pp_p_l_w_purchased_counts", False)
plu_profile_dict["pp_p_l_w_purchased_sum_df"] = getTotalSum(last_week_df, ["pluno"], "amt", "pp_p_l_w_purchased_sum")
plu_profile_dict["pp_p_l_w_purchased_day_ucounts_df"] = getTotalCounts(last_week_df, ["pluno"], "just_date", "pp_p_l_w_purchased_day_ucounts", True)
# last week
# TYPE.1 count/ratio - penetration 衡量品牌名声
plu_profile_dict["pp_p_l_w_has_u_ucounts_df"] = getTotalCounts(last_week_df, ["pluno"], "vipno", "pp_p_l_w_has_u_ucounts", True)
# last month
# TYPE.1 count/ratio - count
plu_profile_dict["pp_p_l_m_purchased_counts_df"] = getTotalCounts(last_month_df, ["pluno"], "id", "pp_p_l_m_purchased_counts", False)
plu_profile_dict["pp_p_l_m_purchased_sum_df"] = getTotalSum(last_month_df, ["pluno"], "amt", "pp_p_l_m_purchased_sum")
plu_profile_dict["pp_p_l_m_purchased_day_ucounts_df"] = getTotalCounts(last_month_df, ["pluno"], "just_date", "pp_p_l_m_purchased_day_ucounts", True)
# last month
# TYPE.1 count/ratio - penetration 衡量品牌名声
plu_profile_dict["pp_p_l_m_has_u_ucounts_df"] = getTotalCounts(last_month_df, ["pluno"], "vipno", "pp_p_l_m_has_u_ucounts", True)


# 
# TYPE.4 complex feature - repeat feature
# 统计repeat buyer的count/ratio
# 1. count:至少两天购买同一I或B或C的人数
# 2. ratio:count / 购买该I或B或C的总人数
plu_profile_dict["pp_plu_repeat_buyer_df"] = getRepeatFeature(feature_extract_df, ["pluno", "vipno"], "pp_plu_repeat_buyer")

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
plu_profile_dict["pp_monthly_p_purchased_counts_df"]["pp_trend_p_purchased_counts_df"] = plu_profile_dict["pp_monthly_p_purchased_counts_df"].apply(getTrend, axis=1)
plu_profile_dict["pp_monthly_p_purchased_sum_df"]["pp_trend_p_purchased_sum_df"] = plu_profile_dict["pp_monthly_p_purchased_sum_df"].apply(getTrend, axis=1)
plu_profile_dict["pp_monthly_p_purchased_day_ucounts_df"]["pp_trend_p_purchased_day_ucounts_df"] = plu_profile_dict["pp_monthly_p_purchased_day_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - penetration 衡量品牌名声
plu_profile_dict["pp_monthly_p_has_u_ucounts_df"]["pp_trend_p_has_u_ucounts_df"] = plu_profile_dict["pp_monthly_p_has_u_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
plu_profile_dict["pp_bias_ratio_p_purchased_counts_df"] = getDeviation(plu_profile_dict["pp_monthly_p_purchased_counts_df"],plu_profile_dict["pp_monthly_p_purchased_counts_agg_df"], ["pluno"], pp_monthly_p_purchased_counts_new_names, "pp_p_purchased_counts")
plu_profile_dict["pp_bias_ratio_p_purchased_sum_df"] = getDeviation(plu_profile_dict["pp_monthly_p_purchased_sum_df"],plu_profile_dict["pp_monthly_p_purchased_sum_agg_df"], ["pluno"], pp_monthly_p_purchased_sum_new_names, "pp_p_purchased_sum")
plu_profile_dict["pp_bias_ratio_p_purchased_day_ucounts_df"] = getDeviation(plu_profile_dict["pp_monthly_p_purchased_day_ucounts_df"],plu_profile_dict["pp_monthly_p_purchased_day_ucounts_agg_df"], ["pluno"], pp_monthly_p_purchased_day_ucounts_new_names, "pp_p_purchased_day_ucounts")
# TYPE.1 count/ratio - penetration 衡量品牌名声
plu_profile_dict["pp_bias_ratio_p_has_u_ucounts_df"] = getDeviation(plu_profile_dict["pp_monthly_p_has_u_ucounts_df"],plu_profile_dict["pp_monthly_p_has_u_ucounts_agg_df"], ["pluno"], pp_monthly_p_has_u_ucounts_new_names, "pp_p_has_u_ucounts")


pp_ohter_list = ["pp_p_l_w_purchased_counts_df", "pp_p_l_w_purchased_sum_df", "pp_p_l_w_purchased_day_ucounts_df",
                "pp_p_l_w_has_u_ucounts_df", "pp_p_l_m_purchased_counts_df", "pp_p_l_m_purchased_sum_df",
                "pp_p_l_m_purchased_day_ucounts_df", "pp_p_l_m_has_u_ucounts_df"]
plu_profile_df = commonMergeDataFrame(plu_profile_dict, pp_ohter_list)
assert isDifferent(feature_extract_df, ["pluno"], plu_profile_df)
plu_profile_df.head()

(3968, 1)
(3968, 75)


,pluno,pp_p_purchased_counts,pp_p_purchased_sum,pp_p_purchased_day_ucounts,pp_first_p_purchased_counts,pp_second_p_purchased_counts,pp_fourth_p_purchased_counts,pp_fifth_p_purchased_counts,pp_trend_p_purchased_counts_df,pp_first_p_purchased_sum,...,pp_p_purchased_counts_bias_divide_mean,pp_p_purchased_sum_bias,pp_p_purchased_sum_bias_divide_std,pp_p_purchased_sum_bias_divide_mean,pp_p_purchased_day_ucounts_bias,pp_p_purchased_day_ucounts_bias_divide_std,pp_p_purchased_day_ucounts_bias_divide_mean,pp_p_has_u_ucounts_bias,pp_p_has_u_ucounts_bias_divide_std,pp_p_has_u_ucounts_bias_divide_mean
0,10000000,1,10.800000,1,0.000000,0.000000,1.000000,0.000000,0.100000,0.000000,...,-1.000000,-3.600000,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
1,10000003,1,5.900000,1,0.000000,0.000000,0.000000,1.000000,0.300000,0.000000,...,0.000000,5.900000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
2,10000004,6,34.800000,6,1.000000,3.000000,1.000000,1.000000,-0.200000,5.800000,...,-0.400000,-3.866667,-0.577350,-0.400000,-0.666667,-0.577350,-0.400000,-0.666667,-0.577350,-0.400000
3,10000005,4,8.000000,4,0.000000,2.000000,1.000000,1.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10000006,20,54.000000,15,10.000000,4.000000,3.000000,3.000000,-2.200000,27.000000,...,-0.470588,-7.200000,-0.704361,-0.470588,-1.000000,-0.577350,-0.250000,-2.000000,-1.154701,-0.500000


### bnd profile

In [9]:
# 812个品牌
bnd_profile_dict = {}

#
# TYPE.1 count/ratio - count
# whold period
# 1. bnd被购买总次数
# 2. bnd被购买总金额
# 3. bnd被购买总天数
bnd_profile_dict["bp_b_purchased_counts_df"] = getTotalCounts(feature_extract_df, ["bndno"], "id", "bp_b_purchased_counts", False)
bnd_profile_dict["bp_b_purchased_sum_df"] = getTotalSum(feature_extract_df, ["bndno"], "amt", "bp_b_purchased_sum")
bnd_profile_dict["bp_b_purchased_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["bndno"], "just_date", "bp_b_purchased_day_ucounts", True)

#
# TYPE.1 count/ratio - count
# monthly
# 1. bnd每月被购买次数
# 2. bnd每月被购买金额
# 3. bnd每月被购买天数
bp_monthly_b_purchased_counts_new_names = getMonthlyNames("bp", "b_purchased_counts")
bp_monthly_b_purchased_sum_new_names = getMonthlyNames("bp", "b_purchased_sum")
bp_monthly_b_purchased_day_ucounts_new_names = getMonthlyNames("bp", "b_purchased_day_ucounts")
bnd_profile_dict["bp_monthly_b_purchased_counts_df"] = getMonthlyCounts(feature_extract_df, ["bndno"], "id", bp_monthly_b_purchased_counts_new_names, False)
bnd_profile_dict["bp_monthly_b_purchased_sum_df"] = getMonthlySum(feature_extract_df, ["bndno"], "amt", bp_monthly_b_purchased_sum_new_names)
bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["bndno"], "just_date", bp_monthly_b_purchased_day_ucounts_new_names, True)

#
# TYPE.1 count/ratio - product diversity
# whold period
bnd_profile_dict["bp_b_has_p_ucounts_df"] =  getTotalCounts(feature_extract_df, ["bndno"], "pluno", "bp_b_has_p_ucounts", True)
#
# TYPE.1 count/ratio - product diversity
# monthly
# 1. 包含的unique的I的数量
# 不知道有无意义
bp_monthly_b_has_p_counts_new_names = getMonthlyNames("bp", "b_has_p_ucounts")
bnd_profile_dict["bp_monthly_b_has_p_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["bndno"], "pluno", bp_monthly_b_has_p_counts_new_names, True)

#
# TYPE.1 count/ratio - penetration 衡量品牌名声
# whold period
# 1. 买过该品牌或品类或商品的不同的买家的人数
bnd_profile_dict["bp_b_has_u_ucounts_df"] = getTotalCounts(feature_extract_df, ["bndno"], "vipno", "bp_b_has_u_ucounts", True)

#
# TYPE.1 count/ratio - penetration 衡量品牌名声
# monthly
# 1. 每月买过该品牌或品类或商品的不同的买家的人数
bp_monthly_b_has_u_ucounts_new_names = getMonthlyNames("bp", "b_has_u_ucounts")
bnd_profile_dict["bp_monthly_b_has_u_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["bndno"], "vipno", bp_monthly_b_has_u_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
bnd_profile_dict["bp_monthly_b_purchased_counts_agg_df"] = getMonthAgg(bnd_profile_dict["bp_monthly_b_purchased_counts_df"], ["bndno"], 1, "bp_monthly_b_purchased_counts")
bnd_profile_dict["bp_monthly_b_purchased_sum_agg_df"] = getMonthAgg(bnd_profile_dict["bp_monthly_b_purchased_sum_df"], ["bndno"], 1, "bp_monthly_b_purchased_sum")
bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_agg_df"] = getMonthAgg(bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_df"], ["bndno"], 1, "bp_monthly_b_purchased_day_ucounts")
# 2. agg: TYPE.1 count/ratio - product diversity
bnd_profile_dict["bp_monthly_b_has_p_ucounts_agg_df"] = getMonthAgg(bnd_profile_dict["bp_monthly_b_has_p_ucounts_df"], ["bndno"], 1, "bp_monthly_b_has_p_ucounts")
# 3. agg: TYPE.1 count/ratio - penetration 衡量品牌名声
bnd_profile_dict["bp_monthly_b_has_u_ucounts_agg_df"] = getMonthAgg(bnd_profile_dict["bp_monthly_b_has_u_ucounts_df"], ["bndno"], 1, "bp_monthly_b_has_u_ucounts")


# TYPE.2 AGG feature - user AGG
# agg_type: 0 - count 1 - nunique 2 - sum
bnd_profile_dict["bp_bnd_user_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["bndno", "vipno"], "bndno", "id", 2, "bp_bnd_user_agg_counts", 0)
bnd_profile_dict["bp_bnd_user_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["bndno", "vipno"], "bndno", "amt", 2, "bp_bnd_user_agg_sum", 2)
bnd_profile_dict["bp_bnd_user_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["bndno", "vipno"], "bndno", "just_date", 2, "bp_bnd_user_agg_day_ucounts", 1)

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week
# TYPE.1 count/ratio - count
bnd_profile_dict["bp_b_l_w_purchased_counts_df"] = getTotalCounts(last_week_df, ["bndno"], "id", "bp_b_l_w_purchased_counts", False)
bnd_profile_dict["bp_b_l_w_purchased_sum_df"] = getTotalSum(last_week_df, ["bndno"], "amt", "bp_b_l_w_purchased_sum")
bnd_profile_dict["bp_b_l_w_purchased_day_ucounts_df"] = getTotalCounts(last_week_df, ["bndno"], "just_date", "bp_b_l_w_purchased_day_ucounts", True)
# last week
# TYPE.1 count/ratio - product diversity
bnd_profile_dict["bp_b_l_w_has_p_ucounts_df"] =  getTotalCounts(last_week_df, ["bndno"], "pluno", "bp_b_l_w_has_p_ucounts", True)
# last week
# TYPE.1 count/ratio - penetration 衡量品牌名声
bnd_profile_dict["bp_b_l_w_has_u_ucounts_df"] = getTotalCounts(last_week_df, ["bndno"], "vipno", "bp_b_l_w_has_u_ucounts", True)
# last month
# TYPE.1 count/ratio - count
bnd_profile_dict["bp_b_l_m_purchased_counts_df"] = getTotalCounts(last_month_df, ["bndno"], "id", "bp_b_l_m_purchased_counts", False)
bnd_profile_dict["bp_b_l_m_purchased_sum_df"] = getTotalSum(last_month_df, ["bndno"], "amt", "bp_b_l_m_purchased_sum")
bnd_profile_dict["bp_b_l_m_purchased_day_ucounts_df"] = getTotalCounts(last_month_df, ["bndno"], "just_date", "bp_b_l_m_purchased_day_ucounts", True)
# last month
# TYPE.1 count/ratio - product diversity
bnd_profile_dict["bp_b_l_m_has_p_ucounts_df"] =  getTotalCounts(last_month_df, ["bndno"], "pluno", "bp_b_l_m_has_p_ucounts", True)
# last month
# TYPE.1 count/ratio - penetration 衡量品牌名声
bnd_profile_dict["bp_b_l_m_has_u_ucounts_df"] = getTotalCounts(last_month_df, ["bndno"], "vipno", "bp_b_l_m_has_u_ucounts", True)

# 
# TYPE.4 complex feature - repeat feature
# 统计repeat buyer的count/ratio
# 1. count:至少两天购买同一I或B或C的人数
# 2. ratio:count / 购买该I或B或C的总人数
bnd_profile_dict["bp_bnd_repeat_buyer_df"] = getRepeatFeature(feature_extract_df, ["bndno", "vipno"], "bp_bnd_repeat_buyer")

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
bnd_profile_dict["bp_monthly_b_purchased_counts_df"]["bp_trend_b_purchased_counts_df"] = bnd_profile_dict["bp_monthly_b_purchased_counts_df"].apply(getTrend, axis=1)
bnd_profile_dict["bp_monthly_b_purchased_sum_df"]["bp_trend_b_purchased_sum_df"] = bnd_profile_dict["bp_monthly_b_purchased_sum_df"].apply(getTrend, axis=1)
bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_df"]["bp_trend_b_purchased_day_ucounts_df"] = bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - product diversity
bnd_profile_dict["bp_monthly_b_has_p_ucounts_df"]["bp_trend_b_has_p_ucounts_df"] = bnd_profile_dict["bp_monthly_b_has_p_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - penetration 衡量品牌名声
bnd_profile_dict["bp_monthly_b_has_u_ucounts_df"]["bp_trend_b_has_u_ucounts_df"] = bnd_profile_dict["bp_monthly_b_has_u_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
bnd_profile_dict["bp_bias_ratio_b_purchased_counts_df"] = getDeviation(bnd_profile_dict["bp_monthly_b_purchased_counts_df"], bnd_profile_dict["bp_monthly_b_purchased_counts_agg_df"], ["bndno"], bp_monthly_b_purchased_counts_new_names, "bp_b_purchased_counts")
bnd_profile_dict["bp_bias_ratio_b_purchased_sum_df"] = getDeviation(bnd_profile_dict["bp_monthly_b_purchased_sum_df"], bnd_profile_dict["bp_monthly_b_purchased_sum_agg_df"], ["bndno"], bp_monthly_b_purchased_sum_new_names, "bp_b_purchased_sum_")
bnd_profile_dict["bp_bias_ratio_b_purchased_day_ucounts_df"] = getDeviation(bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_df"],bnd_profile_dict["bp_monthly_b_purchased_day_ucounts_agg_df"], ["bndno"], bp_monthly_b_purchased_day_ucounts_new_names, "bp_b_purchased_day_ucounts")
# TYPE.1 count/ratio - penetration 衡量品牌名声
bnd_profile_dict["bp_bias_ratio_b_has_u_ucounts_df"] = getDeviation(bnd_profile_dict["bp_monthly_b_has_u_ucounts_df"],bnd_profile_dict["bp_monthly_b_has_u_ucounts_agg_df"], ["bndno"], bp_monthly_b_has_u_ucounts_new_names, "bp_b_has_u_ucounts")


bp_other_list = ["bp_b_l_w_purchased_counts_df", "bp_b_l_w_purchased_sum_df", "bp_b_l_w_purchased_day_ucounts_df",
                "bp_b_l_w_has_p_ucounts_df", "bp_b_l_w_has_u_ucounts_df", "bp_b_l_m_purchased_counts_df",
                "bp_b_l_m_purchased_sum_df", "bp_b_l_m_purchased_day_ucounts_df", "bp_b_l_m_has_p_ucounts_df",
                "bp_b_l_m_has_u_ucounts_df"]
bnd_profile_df = commonMergeDataFrame(bnd_profile_dict, bp_other_list)
assert isDifferent(feature_extract_df, ["bndno"], bnd_profile_df)
bnd_profile_df.head()

(723, 1)
(723, 87)


,bndno,bp_b_purchased_counts,bp_b_purchased_sum,bp_b_purchased_day_ucounts,bp_first_b_purchased_counts,bp_second_b_purchased_counts,bp_fourth_b_purchased_counts,bp_fifth_b_purchased_counts,bp_trend_b_purchased_counts_df,bp_first_b_purchased_sum,...,bp_b_purchased_counts_bias_divide_mean,bp_b_purchased_sum__bias,bp_b_purchased_sum__bias_divide_std,bp_b_purchased_sum__bias_divide_mean,bp_b_purchased_day_ucounts_bias,bp_b_purchased_day_ucounts_bias_divide_std,bp_b_purchased_day_ucounts_bias_divide_mean,bp_b_has_u_ucounts_bias,bp_b_has_u_ucounts_bias_divide_std,bp_b_has_u_ucounts_bias_divide_mean
0,-1.000000,6732,71920.800000,122,1751.000000,1790.000000,1560.000000,1631.000000,-59.000000,18105.610000,...,-0.040776,1513.093333,0.839296,0.085962,0.666667,1.154701,0.021978,-2.666667,-0.529813,-0.009535
1,10003.000000,3,20.500000,3,0.000000,1.000000,1.000000,1.000000,0.300000,0.000000,...,0.500000,9.166667,3.572634,3.235294,0.333333,0.577350,0.500000,0.333333,0.577350,0.500000
2,10005.000000,1,11.000000,1,0.000000,0.000000,1.000000,0.000000,0.100000,0.000000,...,-1.000000,-3.666667,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000,-0.333333,-0.577350,-1.000000
3,10007.000000,41,139.300000,28,14.000000,10.000000,8.000000,9.000000,-1.700000,40.600000,...,-0.156250,-6.966667,-1.989394,-0.190520,-2.666667,-4.618802,-0.347826,-1.333333,-1.154701,-0.160000
4,10012.000000,6,326.000000,3,1.000000,4.000000,0.000000,1.000000,-0.400000,58.000000,...,-0.400000,-58.000000,-0.484885,-0.604167,0.333333,0.577350,0.500000,0.333333,0.577350,0.500000


### dpt profile

In [10]:
# 936个类型
dpt_profile_dict = {}

#
# TYPE.1 count/ratio - count
# whold period
# 1. dpt被购买总次数
# 2. dpt被购买总金额
# 3. dpt被购买总天数
dpt_profile_dict["dp_d_purchased_counts_df"] = getTotalCounts(feature_extract_df, ["dptno"], "id", "dp_d_purchased_counts", False)
dpt_profile_dict["dp_d_purchased_sum_df"] = getTotalSum(feature_extract_df, ["dptno"], "amt", "dp_d_purchased_sum")
dpt_profile_dict["dp_d_purchased_day_ucounts_df"] = getTotalCounts(feature_extract_df, ["dptno"], "just_date", "dp_d_purchased_day_ucounts", True)

#
# TYPE.1 count/ratio - count
# monthly
# 1. dpt每月被购买次数
# 2. dpt每月被购买金额
# 3. dpt每月被购买天数
dp_monthly_d_purchased_counts_new_names = getMonthlyNames("dp", "d_purchased_counts")
dp_monthly_d_purchased_sum_new_names = getMonthlyNames("dp", "d_purchased_sum")
dp_monthly_d_purchased_day_ucounts_new_names = getMonthlyNames("dp", "d_purchased_day_ucounts")
dpt_profile_dict["dp_monthly_d_purchased_counts_df"] = getMonthlyCounts(feature_extract_df, ["dptno"], "id", dp_monthly_d_purchased_counts_new_names, False)
dpt_profile_dict["dp_monthly_d_purchased_sum_df"] = getMonthlySum(feature_extract_df, ["dptno"], "amt", dp_monthly_d_purchased_sum_new_names)
dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["dptno"], "just_date", dp_monthly_d_purchased_day_ucounts_new_names, True)

#
# TYPE.1 count/ratio - product diversity
# whole period
# 1. 包含的unique的I的数量
dpt_profile_dict["dp_d_has_p_ucounts_df"] =  getTotalCounts(feature_extract_df, ["dptno"], "pluno", "dp_d_has_p_ucounts", True)
#
# TYPE.1 count/ratio - product diversity
# monthly
# 1. 包含的unique的I的数量
# 不知道有无意义
dp_monthly_d_has_p_ucounts_new_names = getMonthlyNames("dp", "d_has_p_ucounts") 
dpt_profile_dict["dp_monthly_d_has_p_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["dptno"], "pluno", dp_monthly_d_has_p_ucounts_new_names, True)

#
# TYPE.1 count/ratio - penetration 衡量品类名声
# whold period
# 1. 买过该品牌或品类或商品的不同的买家的人数
dpt_profile_dict["dp_d_has_u_ucounts_df"] = getTotalCounts(feature_extract_df, ["dptno"], "vipno", "dp_d_has_u_ucounts", True)

#
# TYPE.1 count/ratio - penetration 衡量品类名声
# monthly
# 1. 每月买过该品牌或品类或商品的不同的买家的人数
dp_monthly_d_has_u_ucounts_new_names = getMonthlyNames("dp", "d_has_u_ucounts")
dpt_profile_dict["dp_monthly_d_has_u_ucounts_df"] = getMonthlyCounts(feature_extract_df, ["dptno"], "vipno", dp_monthly_d_has_u_ucounts_new_names, True)

# 
# TYPE.2 AGG feature - month AGG
# 针对TYPE.1 中所有的monthly特征，都可以进行此aggregation agg操作包含mean、std、max、median
# 1. agg: TYPE.1 count/ratio - count
dpt_profile_dict["dp_monthly_d_purchased_counts_agg_df"] = getMonthAgg(dpt_profile_dict["dp_monthly_d_purchased_counts_df"], ["dptno"], 1, "dpt_monthly_d_purchased_counts")
dpt_profile_dict["dp_monthly_d_purchased_sum_agg_df"] = getMonthAgg(dpt_profile_dict["dp_monthly_d_purchased_sum_df"], ["dptno"], 1, "dpt_monthly_d_purchased_sum")
dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_agg_df"] = getMonthAgg(dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_df"], ["dptno"], 1, "dpt_monthly_d_purchased_day_ucounts")
# 2. agg: TYPE.1 count/ratio - product diversity
dpt_profile_dict["dp_monthly_d_has_p_ucounts_agg_df"] = getMonthAgg(dpt_profile_dict["dp_monthly_d_has_p_ucounts_df"], ["dptno"], 1, "dp_monthly_d_has_p_ucounts")
# 3. agg: TYPE.1 count/ratio - penetration 衡量品类名声
dpt_profile_dict["dp_monthly_d_has_u_ucounts_agg_df"] = getMonthAgg(dpt_profile_dict["dp_monthly_d_has_u_ucounts_df"], ["dptno"], 1, "dp_monthly_d_has_u_ucounts")

# TYPE.2 AGG feature - user AGG
# agg_type: 0 - count 1 - nunique 2 - sum
dpt_profile_dict["dp_dp_user_agg_counts_df"] = getUIBCAgg(feature_extract_df, ["dptno", "vipno"], "dptno", "id", 2, "dp_dpt_user_agg_counts", 0)
dpt_profile_dict["dp_dp_user_agg_sum_df"] = getUIBCAgg(feature_extract_df, ["dptno", "vipno"], "dptno", "amt", 2, "dp_dpt_user_agg_sum", 2)
dpt_profile_dict["dp_dp_user_agg_day_ucounts_df"] = getUIBCAgg(feature_extract_df, ["dptno", "vipno"], "dptno", "just_date", 2, "dp_dpt_user_agg_day_ucounts", 1)

# 
# TYPE.3 last week / last month feature
# 将时间范围缩小，再进行TYPE.1 和TYPE.2 的特征统计，此时没有 monthly特征和monthly agg特征
# last week
# TYPE.1 count/ratio - count
dpt_profile_dict["dp_d_l_w_purchased_counts_df"] = getTotalCounts(last_week_df, ["dptno"], "id", "dp_d_l_w_purchased_counts", False)
dpt_profile_dict["dp_d_l_w_purchased_sum_df"] = getTotalSum(last_week_df, ["dptno"], "amt", "dp_d_l_w_purchased_sum")
dpt_profile_dict["dp_d_l_w_purchased_day_ucounts_df"] = getTotalCounts(last_week_df, ["dptno"], "just_date", "dp_d_l_w_purchased_day_ucounts", True)
# last week
# TYPE.1 count/ratio - product diversity
dpt_profile_dict["dp_d_l_w_has_p_ucounts_df"] =  getTotalCounts(last_week_df, ["dptno"], "pluno", "dp_d_l_w_has_p_ucounts", True)
# last week
# TYPE.1 count/ratio - penetration 衡量品类名声
dpt_profile_dict["dp_d_l_w_has_u_ucounts_df"] = getTotalCounts(last_week_df, ["dptno"], "vipno", "dp_d_l_w_has_u_ucounts", True)
# last month
# TYPE.1 count/ratio - count
dpt_profile_dict["dp_d_l_m_purchased_counts_df"] = getTotalCounts(last_month_df, ["dptno"], "id", "dp_d_l_m_purchased_counts", False)
dpt_profile_dict["dp_d_l_m_purchased_sum_df"] = getTotalSum(last_month_df, ["dptno"], "amt", "dp_d_l_m_purchased_sum")
dpt_profile_dict["dp_d_l_m_purchased_day_ucounts_df"] = getTotalCounts(last_month_df, ["dptno"], "just_date", "dp_d_l_m_purchased_day_ucounts", True)
# last month
# TYPE.1 count/ratio - product diversity
dpt_profile_dict["dp_d_l_m_has_p_ucounts_df"] =  getTotalCounts(last_month_df, ["dptno"], "pluno", "dp_d_l_m_has_p_ucounts", True)
# last month
# TYPE.1 count/ratio - penetration 衡量品类名声
dpt_profile_dict["dp_d_l_m_has_u_ucounts_df"] = getTotalCounts(last_month_df, ["dptno"], "vipno", "dp_d_l_m_has_u_ucounts", True)


# 
# TYPE.4 complex feature - repeat feature
# 统计repeat buyer的count/ratio
# 1. count:至少两天购买同一I或B或C的人数
# 2. ratio:count / 购买该I或B或C的总人数
dpt_profile_dict["dp_dpt_repeat_buyer_df"] = getRepeatFeature(feature_extract_df, ["dptno", "vipno"], "dp_dpt_repeat_buyer")

#
# TYPE.4 complex feature - trend
# 1. 基于TYPE.1 中的monthly feature，一个monthly feature计算出 一个trend
# TYPE.1 count/ratio - count
dpt_profile_dict["dp_monthly_d_purchased_counts_df"]["dp_trend_d_purchased_counts_df"] = dpt_profile_dict["dp_monthly_d_purchased_counts_df"].apply(getTrend, axis=1)
dpt_profile_dict["dp_monthly_d_purchased_sum_df"]["dp_trend_d_purchased_sum_df"] = dpt_profile_dict["dp_monthly_d_purchased_sum_df"].apply(getTrend, axis=1)
dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_df"]["dp_trend_d_purchased_day_ucounts_df"] = dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - product diversity
dpt_profile_dict["dp_monthly_d_has_p_ucounts_df"]["dp_trend_d_has_p_ucounts_df"] = dpt_profile_dict["dp_monthly_d_has_p_ucounts_df"].apply(getTrend, axis=1)
# TYPE.1 count/ratio - penetration 衡量品牌名声
dpt_profile_dict["dp_monthly_d_has_u_ucounts_df"]["dp_trend_d_has_u_ucounts_df"] = dpt_profile_dict["dp_monthly_d_has_u_ucounts_df"].apply(getTrend, axis=1)

#
# TYPE.4 complex feature - trend
# 2. 基于TYPE.1 中的monthly feature，算最后一个月与前几个月均 值的偏差，并用均值或者标准差normalize
# TYPE.1 count/ratio - count
dpt_profile_dict["dp_bias_ratio_d_purchased_counts_df"] = getDeviation(dpt_profile_dict["dp_monthly_d_purchased_counts_df"], dpt_profile_dict["dp_monthly_d_purchased_counts_agg_df"], ["dptno"], dp_monthly_d_purchased_counts_new_names, "dp_d_purchased_counts")
dpt_profile_dict["dp_bias_ratio_d_purchased_sum_df"] = getDeviation(dpt_profile_dict["dp_monthly_d_purchased_sum_df"], dpt_profile_dict["dp_monthly_d_purchased_sum_agg_df"], ["dptno"], dp_monthly_d_purchased_sum_new_names, "dp_d_purchased_sum_")
dpt_profile_dict["dp_bias_ratio_d_purchased_day_ucounts_df"] = getDeviation(dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_df"],dpt_profile_dict["dp_monthly_d_purchased_day_ucounts_agg_df"], ["dptno"], dp_monthly_d_purchased_day_ucounts_new_names, "dp_d_purchased_day_ucounts")
# TYPE.1 count/ratio - penetration 衡量品牌名声
dpt_profile_dict["dp_bias_ratio_d_has_u_ucounts_df"] = getDeviation(dpt_profile_dict["dp_monthly_d_has_u_ucounts_df"],dpt_profile_dict["dp_monthly_d_has_u_ucounts_agg_df"], ["dptno"], dp_monthly_d_has_u_ucounts_new_names, "dp_d_has_u_ucounts")

dpt_other_list = ["dp_d_l_w_purchased_counts_df", "dp_d_l_w_purchased_sum_df", "dp_d_l_w_purchased_day_ucounts_df",
                 "dp_d_l_w_has_p_ucounts_df", "dp_d_l_w_has_u_ucounts_df", "dp_d_l_m_purchased_counts_df",
                 "dp_d_l_m_purchased_sum_df", "dp_d_l_m_purchased_day_ucounts_df", "dp_d_l_m_has_p_ucounts_df", 
                 "dp_d_l_m_has_u_ucounts_df"]
dpt_profile_df = commonMergeDataFrame(dpt_profile_dict, dpt_other_list)
assert isDifferent(feature_extract_df, ["dptno"], dpt_profile_df)
dpt_profile_df.head()


(838, 1)
(838, 87)


,dptno,dp_d_purchased_counts,dp_d_purchased_sum,dp_d_purchased_day_ucounts,dp_first_d_purchased_counts,dp_second_d_purchased_counts,dp_fourth_d_purchased_counts,dp_fifth_d_purchased_counts,dp_trend_d_purchased_counts_df,dp_first_d_purchased_sum,...,dp_d_purchased_counts_bias_divide_mean,dp_d_purchased_sum__bias,dp_d_purchased_sum__bias_divide_std,dp_d_purchased_sum__bias_divide_mean,dp_d_purchased_day_ucounts_bias,dp_d_purchased_day_ucounts_bias_divide_std,dp_d_purchased_day_ucounts_bias_divide_mean,dp_d_has_u_ucounts_bias,dp_d_has_u_ucounts_bias_divide_std,dp_d_has_u_ucounts_bias_divide_mean
0,10000,90,344.700000,59,25.000000,18.000000,24.000000,23.000000,0.000000,80.800000,...,0.029851,6.166667,0.764626,0.072863,1.666667,2.886751,0.116279,1.666667,2.886751,0.102041
1,10001,16,50.300000,15,2.000000,5.000000,5.000000,4.000000,0.600000,8.900000,...,0.000000,0.033333,0.010099,0.002653,0.333333,0.218218,0.090909,0.333333,0.288675,0.125000
2,10002,61,389.300000,41,15.000000,17.000000,19.000000,10.000000,-1.300000,107.200000,...,-0.411765,7.033333,0.579975,0.073596,-4.333333,-7.505553,-0.382353,-3.000000,-3.000000,-0.250000
3,10008,17,90.400000,16,6.000000,6.000000,5.000000,0.000000,-1.900000,38.000000,...,-1.000000,-30.133333,-3.957610,-1.000000,-5.333333,-9.237604,-1.000000,-4.666667,-8.082904,-1.000000
4,10100,16,191.400000,13,0.000000,6.000000,5.000000,5.000000,1.400000,0.000000,...,0.363636,39.000000,0.944160,1.023622,2.333333,0.927173,0.875000,2.000000,0.755929,0.666667


### Complex

In [11]:
def getMarketShare(input_df, group_by_1, group_by_2, use_col, new_col_name, isUnique):
    res_df = None
    i_df = input_df.copy()
    temp1 = getTotalCounts(i_df, group_by_1, use_col, "_total_count", isUnique)
    temp2 = getTotalCounts(i_df, group_by_2, use_col, "_part_count", isUnique)
    temp2 = temp2.merge(temp1)
    temp2[new_col_name] = temp2["_part_count"] / temp2["_total_count"]
    res_df = temp2[group_by_2 + [new_col_name]]
    return res_df
bd_profile_dict = {}

bd_profile_dict["bcp_c_in_b_count_ratio_df"] = getMarketShare(feature_extract_df, ["bndno"], ["bndno", "dptno"], "id", "bcp_c_in_b_count_ratio", False)
bd_profile_dict["bcp_c_in_b_user_ratio_df"] = getMarketShare(feature_extract_df, ["bndno"], ["bndno", "dptno"], "vipno", "bcp_c_in_b_user_ratio", True)

bd_profile_df = commonMergeDataFrame(bd_profile_dict)
assert isDifferent(feature_extract_df, ["bndno", "dptno"], bd_profile_df)
bd_profile_df.head()


(1789, 2)
(1789, 4)


,bndno,dptno,bcp_c_in_b_count_ratio,bcp_c_in_b_user_ratio
0,-1.000000,10110,0.000743,0.009132
1,-1.000000,10113,0.000149,0.002283
2,-1.000000,10114,0.000446,0.006849
3,-1.000000,10116,0.000446,0.006849
4,-1.000000,10119,0.000446,0.004566


In [12]:
db_profile_dict = {}
db_profile_dict["cbp_b_in_c_count_ratio_df"] = getMarketShare(feature_extract_df, ["dptno"], ["dptno", "bndno"], "id", "cbp_b_in_c_count_ratio", False)
db_profile_dict["cbp_b_in_c_user_ratio_df"] = getMarketShare(feature_extract_df, ["dptno"], ["dptno", "bndno"], "vipno", "cbp_b_in_c_user_ratio", True)

db_profile_df = commonMergeDataFrame(db_profile_dict)
assert isDifferent(feature_extract_df, ["dptno", "bndno"], db_profile_df)
db_profile_df.head()

(1789, 2)
(1789, 4)


,dptno,bndno,cbp_b_in_c_count_ratio,cbp_b_in_c_user_ratio
0,10000,10007.000000,0.377778,0.400000
1,10000,10062.000000,0.477778,0.545455
2,10000,10068.000000,0.144444,0.109091
3,10001,10064.000000,1.000000,1.000000
4,10002,10062.000000,0.016393,0.027778


In [13]:
u_b_profile_df2 = feature_extract_df.groupby(["vipno", "bndno"])["id"].count().reset_index().rename(columns={"id":"count"})
u_d_profile_df2 = feature_extract_df.groupby(["vipno", "dptno"])["id"].count().reset_index().rename(columns={"id":"count"})

ubd_df = feature_extract_df.groupby(["vipno", "bndno","dptno"])["id"].count().reset_index().rename(columns={"id":"count"})
bd_df = bd_profile_df.copy()

udb_df = feature_extract_df.groupby(["vipno", "dptno", "bndno"])["id"].count().reset_index().rename(columns={"id":"count"})
db_df = db_profile_df.copy()

def generate(x, i_ubd_or_udb_df, i_bd_or_db_df, condition_col, ratio_col, agg_type):
    temp_df = i_ubd_or_udb_df.loc[(i_ubd_or_udb_df["vipno"] == x["vipno"]) & (i_ubd_or_udb_df[condition_col] == x[condition_col])]
    temp_df = temp_df.merge(i_bd_or_db_df)
    temp_df["count*count_ratio"] = temp_df["count"] * temp_df[ratio_col]
    if agg_type == 0:
        return temp_df["count*count_ratio"].max()
    elif agg_type == 1:
        return temp_df["count*count_ratio"].mean()

In [14]:
u_b_profile_df2["ubp_similarity_max_1"] = u_b_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = ubd_df, i_bd_or_db_df = bd_df, condition_col = "bndno", ratio_col = "bcp_c_in_b_count_ratio", agg_type=0)
u_b_profile_df2["ubp_similarity_max_2"] = u_b_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = ubd_df, i_bd_or_db_df = bd_df, condition_col = "bndno", ratio_col = "bcp_c_in_b_user_ratio", agg_type=0)
u_b_profile_df2["ubp_similarity_mean_1"] = u_b_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = ubd_df, i_bd_or_db_df = bd_df, condition_col = "bndno", ratio_col = "bcp_c_in_b_count_ratio", agg_type=1)
u_b_profile_df2["ubp_similarity_mean_2"] = u_b_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = ubd_df, i_bd_or_db_df = bd_df, condition_col = "bndno", ratio_col = "bcp_c_in_b_user_ratio", agg_type=1)

u_b_profile_df2.head()


,vipno,bndno,count,ubp_similarity_max_1,ubp_similarity_max_2,ubp_similarity_mean_1,ubp_similarity_mean_2
0,781924,-1.000000,5,0.020053,0.123288,0.010596,0.081431
1,781924,10106.000000,1,0.361702,0.354839,0.361702,0.354839
2,781924,10706.000000,1,0.015038,0.046512,0.015038,0.046512
3,781924,11129.000000,1,0.482759,0.482759,0.482759,0.482759
4,781924,11149.000000,1,0.250000,0.333333,0.250000,0.333333


In [15]:
u_d_profile_df2["udp_similarity_max_1"] = u_d_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = udb_df, i_bd_or_db_df = db_df, condition_col = "dptno", ratio_col = "cbp_b_in_c_count_ratio", agg_type=0)
u_d_profile_df2["udp_similarity_max_2"] = u_d_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = udb_df, i_bd_or_db_df = db_df, condition_col = "dptno", ratio_col = "cbp_b_in_c_user_ratio", agg_type=0)
u_d_profile_df2["udp_similarity_mean_1"] = u_d_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = udb_df, i_bd_or_db_df = db_df, condition_col = "dptno", ratio_col = "cbp_b_in_c_count_ratio", agg_type=1)
u_d_profile_df2["udp_similarity_mean_2"] = u_d_profile_df2.apply(generate, axis=1, i_ubd_or_udb_df = udb_df, i_bd_or_db_df = db_df, condition_col = "dptno", ratio_col = "cbp_b_in_c_user_ratio", agg_type=1)
u_d_profile_df2.head()


,vipno,dptno,count,udp_similarity_max_1,udp_similarity_max_2,udp_similarity_mean_1,udp_similarity_mean_2
0,781924,10113,1,0.200000,0.222222,0.200000,0.222222
1,781924,10130,1,0.459459,0.478261,0.459459,0.478261
2,781924,11302,1,0.153846,0.166667,0.153846,0.166667
3,781924,11531,1,0.045455,0.047619,0.045455,0.047619
4,781924,11532,2,0.285714,0.388889,0.265306,0.277778


In [16]:
# merge ub/ud
u_b_profile_df2 = u_b_profile_df2[["vipno", "bndno", "ubp_similarity_max_1", "ubp_similarity_max_2", "ubp_similarity_mean_1", "ubp_similarity_mean_2"]]
u_d_profile_df2 = u_d_profile_df2[["vipno", "dptno", "udp_similarity_max_1", "udp_similarity_max_2", "udp_similarity_mean_1", "udp_similarity_mean_2"]]

u_b_profile_df = u_b_profile_df.merge(u_b_profile_df2)
u_d_profile_df = u_d_profile_df.merge(u_d_profile_df2)


In [21]:
def formatting(input_df):
    where_are_nan = np.isnan(input_df)  
    where_are_inf = np.isinf(input_df)  
    input_df[where_are_nan] = 0
    input_df[where_are_inf] = 0 
    return input_df.fillna(0)
formatting(user_profile_df).to_csv("./csv/" + directory + "/user_profile_df.csv", index=False)
formatting(u_p_profile_df).to_csv("./csv/" + directory + "/u_p_profile_df.csv", index=False)
formatting(u_b_profile_df).to_csv("./csv/" + directory + "/u_b_profile_df.csv", index=False)
formatting(u_d_profile_df).to_csv("./csv/" + directory + "/u_d_profile_df.csv", index=False)
formatting(plu_profile_df).to_csv("./csv/" + directory + "/plu_profile_df.csv", index=False)
formatting(bnd_profile_df).to_csv("./csv/" + directory + "/bnd_profile_df.csv", index=False)
formatting(dpt_profile_df).to_csv("./csv/" + directory + "/dpt_profile_df.csv", index=False)
if directory != "predict":
    label_df.to_csv("./csv/" + directory + "/label_df.csv", index=False)
print("ok")


ok
